In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
def esa_score(phi, alpha):
    return np.dot(alpha, phi)

def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [3]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

In [4]:
df7 = pd.read_csv('credit_risk_dataset - ethics.csv')

# Initialize LabelEncoder
label_encoder = LabelEncoder()    
    
# Apply label encoding
df7['person_home_ownership'] = label_encoder.fit_transform(df7['person_home_ownership'])
df7['loan_intent'] = label_encoder.fit_transform(df7['loan_intent'])
df7['loan_grade'] = label_encoder.fit_transform(df7['loan_grade'])
df7['cb_person_default_on_file'] = label_encoder.fit_transform(df7['cb_person_default_on_file'])

y = df7['loan_status']
X = df7.drop(columns=['loan_status', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int'])
esa_features = df7[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau_values = df7['CST'].values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])


In [5]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

nb_baseline_preds = np.zeros(len(X))
nb_override_preds = np.zeros(len(X))
nb_penalized_preds = np.zeros(len(X))

b_time = 0
o_time = 0
p_time = 0

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    b_time_s = time.perf_counter()
    # -------- Baseline --------
    model_baseline = GaussianNB()
    model_baseline.fit(X_train, y_train)
    y_val_pred_baseline = model_baseline.predict(X_val)
    nb_baseline_preds[val_idx] = y_val_pred_baseline
    print(f"Fold {fold+1} - Naive Bayes Baseline:")
    evaluate_classification(y_val, y_val_pred_baseline)
    b_time_e = time.perf_counter()
    b_time = b_time + (b_time_e - b_time_s)
    
    o_time_s = time.perf_counter()
    # -------- Override --------
    phi_val = esa_features[val_idx]
    tau_val = tau_values[val_idx]
    esa_vals = np.array([esa_score(phi, alpha) for phi in phi_val])
    moral_preds = (esa_vals >= tau_val).astype(int)
    nb_override_preds[val_idx] = moral_preds
    print(f"Fold {fold+1} - Naive Bayes ESA Override:")
    evaluate_classification(y_val, moral_preds)
    o_time_e = time.perf_counter()
    o_time = o_time + (o_time_e - o_time_s)

    p_time_s = time.perf_counter()
    # -------- Penalized -------- (reweighting the train set)
    phi_train = esa_features[train_idx]
    tau_train = tau_values[train_idx]
    moral_penalty = np.array([(tau - esa_score(phi, alpha))**2 for phi, tau in zip(phi_train, tau_train)])
    sample_weights = np.clip(1 + 5 * moral_penalty, 1, 10)
    
    model_penalized = GaussianNB()
    model_penalized.fit(X_train, y_train, sample_weight=sample_weights)
    y_val_pred_penalized = model_penalized.predict(X_val)
    nb_penalized_preds[val_idx] = y_val_pred_penalized
    print(f"Fold {fold+1} - Naive Bayes ESA Penalized:")
    evaluate_classification(y_val, y_val_pred_penalized)
    p_time_e = time.perf_counter()
    p_time = p_time + (p_time_e - p_time_s)
    
print("\n--- Final Evaluation (Naive Bayes Baseline) ---")
evaluate_classification(y, nb_baseline_preds)
print("Accuracy:", accuracy_score(y, nb_baseline_preds))

print("\n--- Final Evaluation (Naive Bayes ESA Override) ---")
evaluate_classification(y, nb_override_preds)
print("Accuracy:", accuracy_score(y, nb_override_preds))

print("\n--- Final Evaluation (Naive Bayes ESA Penalized) ---")
evaluate_classification(y, nb_penalized_preds)
print("Accuracy:", accuracy_score(y, nb_penalized_preds))

print("time baseline:", b_time)
print("time override:", o_time)
print("time penalized:", p_time)

Fold 1 - Naive Bayes Baseline:

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.83      0.86      5095
           1       0.52      0.66      0.58      1422

    accuracy                           0.79      6517
   macro avg       0.71      0.74      0.72      6517
weighted avg       0.81      0.79      0.80      6517


Confusion Matrix:
[[4226  869]
 [ 488  934]]
Fold 1 - Naive Bayes ESA Override:

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      5095
           1       0.31      0.33      0.32      1422

    accuracy                           0.69      6517
   macro avg       0.56      0.56      0.56      6517
weighted avg       0.70      0.69      0.70      6517


Confusion Matrix:
[[4049 1046]
 [ 957  465]]
Fold 1 - Naive Bayes ESA Penalized:

Classification Report:
              precision    recall  f1-score   support

           0       0.90  

In [12]:
metrics = evaluate_classification(y, nb_penalized_preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.84      0.86     25473
           1       0.52      0.64      0.57      7108

    accuracy                           0.79     32581
   macro avg       0.71      0.74      0.72     32581
weighted avg       0.81      0.79      0.80     32581


Confusion Matrix:
[[21316  4157]
 [ 2564  4544]]
{'accuracy': 0.7937141278659341, 'precision': 0.5222388231237789, 'recall': 0.6392796848621272, 'f1_score': 0.5748624201404263, 'roc_auc': 0.7380436425331325}


In [13]:
metrics = evaluate_classification(y, nb_override_preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.80      0.80     25473
           1       0.31      0.32      0.31      7108

    accuracy                           0.69     32581
   macro avg       0.56      0.56      0.56     32581
weighted avg       0.70      0.69      0.70     32581


Confusion Matrix:
[[20377  5096]
 [ 4852  2256]]
{'accuracy': 0.6946686719253553, 'precision': 0.3068552774755169, 'recall': 0.317388857625211, 'f1_score': 0.31203319502074683, 'roc_auc': 0.5586669487356613}


In [14]:
metrics = evaluate_classification(y, nb_baseline_preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.84      0.86     25473
           1       0.52      0.64      0.58      7108

    accuracy                           0.79     32581
   macro avg       0.71      0.74      0.72     32581
weighted avg       0.81      0.79      0.80     32581


Confusion Matrix:
[[21277  4196]
 [ 2546  4562]]
{'accuracy': 0.7930695804303122, 'precision': 0.5208951815482987, 'recall': 0.6418120427687113, 'f1_score': 0.575066179251229, 'roc_auc': 0.7385443050572642}
